In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML
cf.set_config_file(sharing="public",theme="white",offline=True)

In [2]:
df_hidro = pd.read_csv("hidrocarburos_limpio.csv", index_col="ID")

In [3]:
df_hidro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132980 entries, 123 to 134111
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   du                  132980 non-null  object 
 1   trdu                132980 non-null  object 
 2   empresa_informante  132980 non-null  object 
 3   tipo                132980 non-null  object 
 4   tipo_tramo          132980 non-null  object 
 5   diametro            132980 non-null  float64
 6   prof                132980 non-null  float64
 7   material            132980 non-null  object 
 8   revestim            132980 non-null  object 
 9   geojson             132980 non-null  object 
 10  longitud            132980 non-null  float64
dtypes: float64(3), object(8)
memory usage: 12.2+ MB


In [4]:
# La empresa SINOPEC ARGENTINA y CGC ENERGIA declaran exactamente la misma cantidad de ductos, asumo que están duplicados
# los datos por algún motivo desconocido.

df_hidro.drop(df_hidro[df_hidro["empresa_informante"] == "CGC ENERGIA SAU"].index,inplace=True)

# La empresa PAN AMERICAN ENERGY SL y PAN AMERICAN ENERGY (SUCURSAL ARGENTINA) LLC las considero como una única empresa 
# a los fines prácticos.

df_hidro.loc[df_hidro["empresa_informante"] == "PAN AMERICAN ENERGY SL","empresa_informante"] = "PAN AMERICAN ENERGY (SUCURSAL ARGENTINA) LLC"

# La empresa VISTA ENERGY ARGENTINA SAU y VISTA OIL & GAS ARGENTINA SAU las considero como una única empresa 
# a los fines prácticos.

df_hidro.loc[df_hidro["empresa_informante"] == "VISTA ENERGY ARGENTINA SAU","empresa_informante"] = "VISTA OIL & GAS ARGENTINA SAU"

# Agrego una columna que determine si es Troncal o No Troncal
df_hidro.loc[(df_hidro["diametro"] < 20), "troncal"] = "NO TRONCAL"
df_hidro.loc[(df_hidro["diametro"] >= 20), "troncal"] = "TRONCAL"

In [5]:
empresa_tipo = df_hidro.pivot_table(index="empresa_informante", columns="tipo", values="longitud", aggfunc='sum', margins = "total").fillna(0)
empresa_tipo = empresa_tipo.sort_values(by="All", ascending=True)
empresa_tipo = empresa_tipo.reset_index()
empresa_tipo_primeros = empresa_tipo.tail(13)
empresa_tipo_otros = empresa_tipo.iloc[:-13].sum()
ultima_fila = pd.DataFrame(empresa_tipo_otros).transpose()
ultima_fila["empresa_informante"] = "Otros"
ultima_empresa_tipo = pd.concat([empresa_tipo_primeros,ultima_fila])
ultima_empresa_tipo = ultima_empresa_tipo.sort_values(by = "All", ascending=True)
ultima_empresa_tipo=ultima_empresa_tipo.reset_index(drop=True)
ultima_empresa_tipo = ultima_empresa_tipo.infer_objects()


In [6]:
ultima_empresa_tipo

tipo,empresa_informante,ACUEDUCTO,GASODUCTO,OLEODUCTO,OTROS,POLIDUCTO,All
0,PETROLERA ENTRE LOMAS S.A.,118.152701,202.042264,186.644824,0.000000,0.000000,506.839789
1,TOTAL AUSTRAL S.A.,4.811562,425.919574,173.003681,4.007238,92.439845,700.181900
2,CAPEX S.A.,63.077838,87.133580,303.604606,0.000000,399.488792,853.304815
3,OILSTONE ENERGIA S.A.,290.599075,764.175128,741.236677,0.000000,0.000000,1796.010880
4,VISTA OIL & GAS ARGENTINA SAU,290.534204,620.848479,931.154510,0.000000,0.000000,1842.537193
5,PETROBRAS ARGENTINA S.A.,0.000000,752.884137,1082.999753,0.000000,24.566513,1860.450402
6,CHEVRON ARGENTINA S.R.L.,579.367241,161.103103,1333.368952,0.000000,0.000000,2073.839297
7,Otros,353.598251,1452.095866,1586.549930,295.955159,112.018391,3800.217597
8,TECPETROL S.A.,694.404638,1297.304333,1858.782047,56.070616,120.448344,4027.009978
9,PLUSPETROL S.A.,1228.829532,1404.666751,1831.484114,0.000000,0.792063,4465.772460


In [25]:
fig =px.bar(ultima_empresa_tipo.iloc[:-1,:-1],y =ultima_empresa_tipo.iloc[:-1,:-1]["empresa_informante"],x=ultima_empresa_tipo.iloc[:-1,:-1].columns[1:], orientation="h")


fig.update_layout(
    barmode='stack',
    xaxis=dict(title='Kilómetros',tickformat='.0f'),
    title='Longitud de ductos por Empresa y Tipo',
    yaxis=dict(title="Empresa Informante"),
    height=600)

fig.write_html("Longitud de ductos por empresa y tipo.html")
fig.show()

In [8]:
df_hidro_reducida = df_hidro.drop(columns=["du","trdu","tipo","tipo_tramo","diametro","prof","material","revestim","geojson"])

In [9]:
tienen_cantidad_troncal_por_empresa = df_hidro_reducida.groupby(["empresa_informante","troncal"]).sum().reset_index()
tienen_cantidad_troncal_por_empresa['longitud_total'] = tienen_cantidad_troncal_por_empresa.groupby('empresa_informante')['longitud'].transform('sum')
tienen_cantidad_troncal_por_empresa['longitud_pct'] = tienen_cantidad_troncal_por_empresa['longitud'] / tienen_cantidad_troncal_por_empresa['longitud_total'] * 100

tienen_cantidad_troncal_por_empresa=tienen_cantidad_troncal_por_empresa[tienen_cantidad_troncal_por_empresa["longitud_pct"] !=100]
tienen_cantidad_troncal_por_empresa=pd.concat([tienen_cantidad_troncal_por_empresa,pd.DataFrame({'empresa_informante': ['OTROS'],'troncal': ['NO TRONCAL'],'longitud': [0],'longitud_total': [0],'longitud_pct': [100]})])
          

In [27]:
fig = px.bar(tienen_cantidad_troncal_por_empresa, y="empresa_informante", x="longitud_pct", color="troncal", orientation="h",
             labels={'longitud_pct':'Porcentaje de longitud'}, text='longitud_pct')

fig.update_traces(texttemplate='%{text:.1f}%', textposition='inside')
fig.update_layout(
    barmode='stack',
    xaxis=dict(title='Porcentaje', ticksuffix='%'),
        title='Porcentaje por longitud por ductos Troncales vs No Troncales',
    height=600
)

fig.show()
fig.write_html("Porcentaje por longitud por ductos troncales vs no troncales.html")

In [11]:
# Hago lo mismo pero para ductos troncales, es decir mayor a 20 pulgadas.

df_hidro_trocal = df_hidro[df_hidro["diametro"] >= 20]
empresa_tipo2 = df_hidro_trocal.pivot_table(index="empresa_informante", columns="tipo", values="longitud", aggfunc='sum', margins = "total").fillna(0)
empresa_tipo2 = empresa_tipo2.sort_values(by="All", ascending=True)
empresa_tipo2 = empresa_tipo2.reset_index()
empresa_tipo_primeros2 = empresa_tipo2.tail(13)
empresa_tipo_otros2 = empresa_tipo2.iloc[:-13].sum()
ultima_fila2 = pd.DataFrame(empresa_tipo_otros2).transpose()
ultima_fila2["empresa_informante"] = "Otros"
ultima_empresa_tipo2 = pd.concat([empresa_tipo_primeros2,ultima_fila2])
ultima_empresa_tipo2 = ultima_empresa_tipo2.sort_values(by = "All", ascending=True)
ultima_empresa_tipo2=ultima_empresa_tipo2.reset_index(drop=True)
ultima_empresa_tipo2 = ultima_empresa_tipo2.infer_objects()

In [28]:
fig = px.bar(ultima_empresa_tipo2.iloc[:-1,:-1],y =ultima_empresa_tipo2.iloc[:-1,:-1]["empresa_informante"],x=ultima_empresa_tipo2.iloc[:-1,:-1].columns[1:], orientation="h")


fig.update_layout(
    barmode='stack',
    xaxis=dict(title='Kilómetros',tickformat='.0f'),
    title='Longitud de ductos troncales por Empresa y Tipo',
    yaxis=dict(title="Empresa Informante"),
    height=600)

fig.write_html("Longitud de ductos troncales por empresa y tipo.html")
fig

In [29]:
fig = px.pie(df_hidro_trocal,names="tipo",values="longitud") 


fig.update_layout(
    title='Porcentaje de longitud por tipo de ductos',
    
)
fig.write_html("Porcentaje de longitud total por tipo de ducto.html")
fig

In [30]:
fig = px.pie(df_hidro_trocal[df_hidro_trocal["material"] != "SIN DATO"],names="material",values="longitud")

fig.update_layout(
    title='Porcentaje de longitud por material',
)

fig.write_html("Porcentaje de longitud por tipo de material.html")
fig

In [31]:
fig = px.pie(df_hidro_trocal[df_hidro_trocal["revestim"] != "SIN DATOS"],names="revestim",values="longitud")


fig.update_layout(
    title='Porcentaje de longitud por revestimientos',
)

fig.write_html("Porcentaje de longitud por tipo de recubrimiento.html")
fig

In [33]:
fig = px.sunburst(df_hidro_trocal, path=['tipo_tramo', 'tipo'], values='longitud')


fig.update_layout(
    title='Porcentaje de longitud por tipo de ducto y por tipo de tramo',
)
fig.write_html("Proporción en longitud por tipo de ducto y tipo de tramo.html")

fig

In [32]:
df_grouped = df_hidro_trocal.groupby(['empresa_informante', 'tipo_tramo']).sum().reset_index()
df_grouped = df_grouped.drop(columns=["du","trdu","tipo","prof","material","revestim","diametro","geojson","troncal"])
df_grouped['longitud_total'] = df_grouped.groupby('empresa_informante')['longitud'].transform('sum')
df_grouped['longitud_pct'] = df_grouped['longitud'] / df_grouped['longitud_total'] * 100

fig = px.bar(df_grouped, y="empresa_informante", x="longitud_pct", color="tipo_tramo", orientation="h",
             labels={'longitud_pct':'Porcentaje de longitud'}, text='longitud_pct')

fig.update_traces(texttemplate='%{text:.1f}%', textposition='inside')
fig.update_layout(
    barmode='stack',
    xaxis=dict(title='Porcentaje', ticksuffix='%'),
    title='Proporción de longitud por tipo de ducto y empresa informante',
    height=600,
    yaxis=dict(title='Empresa informante')
)

fig.write_html("Proporción en longitud por tipo de tramo y empresa informante.html")
fig.show()

In [18]:
df_hidro_troncal = df_hidro[df_hidro["diametro"] >= 20]
df_hidro_troncal['deciles'] = pd.qcut(df_hidro_troncal['longitud'], q=10, labels=False)

In [19]:
df_grouped_2 = df_hidro_troncal.groupby(['empresa_informante',"deciles","tipo_tramo"]).sum().reset_index()
df_grouped_2 = df_grouped_2.drop(columns=["du","trdu","tipo","prof","material","revestim","diametro","geojson","troncal"])

In [34]:
import pandas as pd
import plotly.graph_objects as go

# Unir las columnas 'tipo' y 'tipo_tramo' para obtener las etiquetas únicas
all_labels = pd.concat([df_grouped_2["empresa_informante"], df_grouped_2["deciles"],df_grouped_2["tipo_tramo"]]).unique().tolist()


# Crear los enlaces del diagrama de Sankey
source = []
target = []
value = []

# Añadir enlaces entre 'tipo' y 'tipo_tramo'
source.extend(df_grouped_2["empresa_informante"].map(lambda x: all_labels.index(x)))
target.extend(df_grouped_2["tipo_tramo"].map(lambda x: all_labels.index(x)))
value.extend(df_grouped_2["longitud"])

# Añadir enlaces entre 'tipo_tramo' y 'material'
source.extend(df_grouped_2["tipo_tramo"].map(lambda x: all_labels.index(x)))
target.extend(df_grouped_2["deciles"].map(lambda x: all_labels.index(x)))
value.extend(df_grouped_2["longitud"])



# Crear un objeto de figura Sankey

fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=all_labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
)])

fig.update_layout(
    title='Empresa informante - Tipo de ducto - Longitud en deciles',
    
)

# Mostrar el diagrama

fig.write_html("Empresa informante - tipo de ducto - longitud en deciles.html")
fig.show()

In [21]:
df_hidro

,du,trdu,empresa_informante,tipo,tipo_tramo,diametro,prof,material,revestim,geojson,longitud,troncal
ID,,,,,,,,,,,,
123,ALTO LAS HORMIGAS,ALTO LAS HORMIGAS-LAS BASES,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,GASODUCTO,ENTERRADO,6.000,1.2,ACERO,POLIURETANO,"{""type"":""MultiLineString"",""coordinates"":[[[-67...",17.765197,NO TRONCAL
124,DEL T-25,DEL T-25,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.117905,NO TRONCAL
125,DEL T-47,DEL T-47,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.181930,NO TRONCAL
126,DEL TR-1014,DEL TR-1014,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.708452,NO TRONCAL
127,DEL T-36,DEL T-36,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.344256,NO TRONCAL
...,...,...,...,...,...,...,...,...,...,...,...,...
134107,ACUEDUCTO PIA CL A SATÃLITE DE INY NÂ° 20,DUCAPIACLASAT16/T2,YPF S.A.,ACUEDUCTO,ENTERRADO,4.000,1.2,ACERO,SIN DATOS,"{""type"":""MultiLineString"",""coordinates"":[[[-67...",0.809833,NO TRONCAL
134108,ACUEDUCTO DE AGUA DULCE,S/E,YPF S.A.,ACUEDUCTO,AEREO,6.000,0.0,ERFV,SIN DATOS,"{""type"":""MultiLineString"",""coordinates"":[[[-68...",18.292221,NO TRONCAL
134109,TRONCAL AGUA PRODUCCION SAT-VI-1-PIAS-4 - DERI...,SIN DATO,YPF S.A.,ACUEDUCTO,AEREO,6.000,0.0,ACERO,SIN DATOS,"{""type"":""MultiLineString"",""coordinates"":[[[-68...",2.645050,NO TRONCAL


In [35]:
fig = px.scatter(df_hidro_troncal, 
                 x="diametro", 
                 y='longitud', 
                 color='tipo',  # Diferenciar los puntos según la edad
                 #size='prof',   # Opcional: tamaño de los puntos según la edad
                 title='Relación entre diámetro y longitud con tipo de ducto',
                 
                )
fig.write_html("Tipo de ducto según relación entre diámetro y longitud.html")
fig.show()